# Pythonで作る2次元変位解析FEM
## 1.はじめに
### 1.1 Pythonとは
Pythonは、いわゆるLL(Lightweight Language)のひとつである。ここでのLightweightというのは、動作が軽量という意味ではなく、習得・学習・使用が容易(軽量)であるということから付けられている。Pythonはその学習のしやすさ、それに伴うユーザーの多さ、パッケージの豊富さから、近年のAIブームにおける使用言語のデファクトスタンダードとなっている。もしあなたが今Pythonを使ったことがなかったとしても、これを機にPythonを始めてみることをおすすめする。

### 1.2 有限要素法とは
有限要素法(FEM : Finite Element Method)とは、解析したい物や場を細かい要素に区切り、その細かい要素について計算した結果を重ね合わせて全体を求める手法である。流れ場の解析や構造物の応力解析などに広く使われる手法である。本文書では、2次元の構造物を対象とした応力解析問題について有限要素法を用いる。

## 2.定式化
弾性体の支配方程式は、いわゆるフックの法則である以下の式で表される。
$$
F=KU
$$

この式は解析対象全体についての式だが、これを有限要素に分解(離散化)する。1要素についての支配方程式は以下である。
$$
F_e=K_eU_e
$$

ただし、系全体の合成マトリクス$K$は、各要素の剛性マトリクス$K_e$の重ね合わせで表される。
$$
K=\sum_e^{\mathrm{All\;Elements}}K_e
$$

ここで、$K_e$は以下のようになる。
$$
K_e=\iint_{\Omega_e} B^TDBdet(J)d\Omega_e
$$

一般に積分をプログラム上で行うことは困難であるから、数値積分によって計算し易い形式に近似する。数値積分法の1つであるガウス・ルジャンドル求積を適用して、
$$
K_e=\sum_{i=1}^4 \sum_{j=1}^4 \omega_i \omega_j B^T(\xi_{i,j},\eta_{i,j})DB(\xi_{i,j},\eta_{i,j})det(J(\xi_{i,j},\eta_{i,j}))
$$

$$
\omega_i=\omega_j=1 \;\;(i,j=1,2,3,4)
$$

$$
(\xi_{i,j},\eta_{i,j})=
(\pm \frac{1}{\sqrt 3},\pm \frac{1}{\sqrt 3}),
(\pm \frac{1}{\sqrt 3},\mp \frac{1}{\sqrt 3})
$$

となる。$(\xi_{i,j},\eta_{i,j})$はガウス・ルジャンドル求積の積分点の正規化座標での座標値である。

今、上式においての未知数は$B$、$D$、$J$である。それらの具体形を以下に示す。
ひずみ$\epsilon$は以下のように表される。
$$
\epsilon=\left( \begin{array}{} 
\epsilon_x \\
\epsilon_y \\
\gamma_{xy}
\end{array} \right)
=\left( \begin{array}{}
\frac{\partial u}{\partial x} \\
\frac{\partial v}{\partial y} \\
\frac{\partial v}{\partial x}+
\frac{\partial u}{\partial y}
\end{array} \right)
=\left( \begin{array}{}
\frac{\partial}{\partial y} & 0 \\
0 & \frac{\partial}{\partial x} \\
\frac{\partial}{\partial x} & \frac{\partial}{\partial y}
\end{array} \right)
\left( \begin{array}{}
u \\
v
\end{array} \right)
=Au
$$

また$B$マトリクスは上記の$A$を用いて以下のように表される。

$$
B=\left( \begin{array}{}
B_1 & B_2 & B_3 & B_4 
\end{array} \right)
$$

$$
B_n=AN_n=\left( \begin{array}{}
\frac{\partial}{\partial y} & 0 \\
0 & \frac{\partial}{\partial x} \\
\frac{\partial}{\partial x} & \frac{\partial}{\partial y}
\end{array} \right)
\left( \begin{array}{}
N_n & 0 \\ 
0 & N_n
\end{array} \right)
=\left( \begin{array}{}
\frac{\partial N_n}{\partial y} & 0 \\
0 & \frac{\partial N_n}{\partial x} \\
\frac{\partial N_n}{\partial x} & \frac{\partial N_n}{\partial y}
\end{array} \right)
$$

$$
(n=1,2,3,4)
$$

ここで、$B_n$の成分は偏微分の連鎖則から以下のように表される。
$$
\frac{\partial N_n}{\partial \xi}=
\frac{\partial x}{\partial \xi} \frac{\partial N_n}{\partial x}+
\frac{\partial y}{\partial \xi} \frac{\partial N_n}{\partial y}
$$

$$
\frac{\partial N_n}{\partial \eta}=
\frac{\partial x}{\partial \eta} \frac{\partial N_n}{\partial x}+
\frac{\partial y}{\partial \eta} \frac{\partial N_n}{\partial y}
$$


以上の関係を行列形式で表せば、

$$
\left( \begin{array}{}
\frac{\partial N_n}{\partial \xi} \\ 
\frac{\partial N_n}{\partial \eta} 
\end{array} \right)=
\left( \begin{array}{}
\frac{\partial x}{\partial \xi} & \frac{\partial y}{\partial \xi} \\ 
\frac{\partial x}{\partial \eta} & \frac{\partial y}{\partial \eta}
\end{array} \right)
\left( \begin{array}{}
\frac{\partial N_n}{\partial \xi} \\ 
\frac{\partial N_n}{\partial \eta} 
\end{array} \right)=
J
\left( \begin{array}{}
\frac{\partial N_n}{\partial \xi} \\ 
\frac{\partial N_n}{\partial \eta} 
\end{array} \right)
$$

となる。ここで$J$はヤコビマトリクスである。

$$
x = \sum_{n=1}^4 N_n x_n ,\;\;
y = \sum_{n=1}^4 N_n y_n
$$

$$
\frac{\partial x}{\partial \xi} = \sum_{n=1}^4 \frac{\partial N_n}{\partial \xi} x_n
,\;\;
\frac{\partial x}{\partial \eta} = \sum_{n=1}^4 \frac{\partial N_n}{\partial \eta} x_n
$$

$$
\frac{\partial y}{\partial \xi} = \sum_{n=1}^4 \frac{\partial N_n}{\partial \xi} y_n
,\;\;
\frac{\partial y}{\partial \eta} = \sum_{n=1}^4 \frac{\partial N_n}{\partial \eta} y_n
$$

$$
J=
\left( \begin{array}{}
\frac{\partial N_1}{\partial \xi} & 
\frac{\partial N_2}{\partial \xi} &
\frac{\partial N_3}{\partial \xi} &
\frac{\partial N_4}{\partial \xi} \\ 
\frac{\partial N_1}{\partial \eta} &
\frac{\partial N_2}{\partial \eta} &
\frac{\partial N_3}{\partial \eta} &
\frac{\partial N_4}{\partial \eta} 
\end{array} \right)
\left( \begin{array}{}
x_1 & y_1 \\ 
x_2 & y_2 \\ 
x_3 & y_3 \\ 
x_4 & y_4 
\end{array} \right)
$$

$$
\left( \begin{array}{}
N_1 \\ 
N_2 \\ 
N_3 \\ 
N_4 
\end{array} \right)=
\frac{1}{4}
\left( \begin{array}{}
(1-\xi)(1-\eta) \\ 
(1+\xi)(1-\eta) \\
(1+\xi)(1+\eta) \\
(1-\xi)(1+\eta) \\
\end{array} \right)
$$

$$
\frac{\partial N_1}{\partial \xi}=-\frac{1-\eta}{4} ,\;\;
\frac{\partial N_2}{\partial \xi}=\frac{1-\eta}{4} ,\;\;
\frac{\partial N_3}{\partial \xi}=\frac{1+\eta}{4} ,\;\;
\frac{\partial N_4}{\partial \xi}=-\frac{1+\eta}{4}
$$

$$
\frac{\partial N_1}{\partial \eta}=-\frac{1-\xi}{4} ,\;\;
\frac{\partial N_2}{\partial \eta}=-\frac{1+\xi}{4} ,\;\;
\frac{\partial N_3}{\partial \eta}=\frac{1+\xi}{4} ,\;\;
\frac{\partial N_4}{\partial \eta}=\frac{1-\xi}{4}
$$

$D$マトリクスは以下である。これは平面応力状態を仮定した際の式である。平面応力状態は薄い平板が面方向に荷重を受ける状態であり、板に垂直方向($z$方向)の応力を0とみなす仮定である。

$$
D=\frac{E}{1-\nu^2}
\left( \begin{array}{}
1 & \nu & 0\\ 
\nu & 1 & 0 \\
0 & 0 & \frac{1-\nu}{2} 
\end{array} \right)
$$

$E$はヤング率、$\nu$はポアソン比である。
参考として、平面ひずみ状態での$D$マトリクスを以下に示す。平面ひずみ状態では、$z$方向を非常に長いと仮定して$z$方向のひずみを0とみなす。

$$
D=\frac{E}{(1+\nu)(1-2\nu)}
\left( \begin{array}{}
1-\nu & \nu & 0\\ 
\nu & 1-\nu & 0 \\
0 & 0 & \frac{1-2\nu}{2} 
\end{array} \right)
$$

## 3.実装

ここからは前出の定式化に登場した計算をFEMクラスのfemメソッドに追加していく。$K$マトリクスやその算出に必要なマトリクスはFEMクラスの内部メソッドとして実装して計算の見通しを良くする。

FEMクラス内のfemの計算メソッドは以下である。

```python
class FEM:
    # 略
    def fem(self):
        K_sp=self._Kmat_sp()

        K_free_sp = K_sp[self.free_nodes].T[
            self.free_nodes].T
        
        U = np.zeros(self.all_vector_count)
        solve_time=time.time()

        U[self.free_nodes] = spsolve(K_free_sp, sp.lil_matrix(
            self.F[self.free_nodes]).tobsr().T)
        print(f"solve time {time.time()-solve_time:.3f} [sec]")

        self.U = U
```

疎行列で計算をしているためやや複雑に見えるが、本質としてはKを定義してF=KUを解いてUを求めているだけである。
K_spに全体の剛性マトリクスを定義し、与えられた外力FについてF=KUをspsolveで解いている。なお拘束されている点では変位が0であるため、拘束されていない点をfree_nodesで指定して、対応するノードについてのみ計算している。

剛性マトリクスKを求めるメソッドを定式化の通り定義していく。Kは各要素の剛性マトリクスKeの重ね合わせで表現される。疎行列で実装したものは_Kmat_spというメソッドで、計算にはこちらを使用しているが、説明のため疎行列で実装していないメソッド_Kmatを見ていく。実装は以下のようになっている。

```python
class FEM:
    # 略
    def _Kmat(self):
        K = np.zeros([self.all_vector_count, self.all_vector_count])
        for y in tqdm(range(self.ny)):
            for x in range(self.nx):
                Ke = self._Kemat(x, y)
                top1 = (self.ny+1)*x+y
                top2 = (self.ny+1)*(x+1)+y
                elem = [2*top1, 2*top1+1, 2*top2, 2*top2+1,
                        2*top2+2, 2*top2+3, 2*top1+2, 2*top1+3]
                for index, one_elem in enumerate(elem):
                    K[elem, one_elem] += Ke[index]

        return K
```

ある(x,y)番目の要素についての剛性マトリクスKeを求め、全体の剛性マトリクスKについて足し合わせる、という計算を、2重のforループで全要素にわたって行っていることがわかる。疎行列で実装したものについてもほぼ同様の計算をしている。

続いて、そのKeを実装する。定式化から、Keの計算式は

$$
K_e=\sum_{i=1}^4 \sum_{j=1}^4 \omega_i \omega_j B^T(\xi_{i,j},\eta_{i,j})DB(\xi_{i,j},\eta_{i,j})det(J(\xi_{i,j},\eta_{i,j}))
$$

Bマトリクス、Dマトリクス、Jマトリクスを計算して、それらを用いてKeに上式を足し合わせていくことでKeを求めている。

```python
class FEM:
    # 略
    def _Kemat(self, x, y):
        Ke = np.zeros((8, 8))

        for n in range(4):
            xi, eta = self.point[n]
            w = self.weight[n]
            dNdxi = self._dNdxi(eta)
            dNdeta = self._dNdeta(xi)
            J = self._Jmat(x, y, dNdxi, dNdeta)
            B = self._Bmat(J, dNdxi, dNdeta)
            Ke += w*(B.T @ self._Dmat) @ B*np.linalg.det(J)
        return Ke
```

別途Bマトリクス、Dマトリクス、Jマトリクスを実装する必要がある。


Dマトリクスは弾性マトリクスと呼ばれ、ヤング率$E$、ポアソン比$\nu$のみで定まる各要素で共通なマトリクスである。このためDマトリクスは逐一計算する必要がなく、一度計算したものを使いまわせばよい。本実装ではクラスFEMの初期化の際に与えられたヤング率とポアソン比からDを計算している。また平面応力状態の場合について計算している。

```python
class FEM:
    def __init__(self, nx, ny, mesh_size, fix_nodes, F, E0, nu):
    # 略
    # plane stress condition
    self._Dmat = E0 * np.array([[1, nu, 0],
                            [nu, 1, 0],
                            [0, 0, (1-nu)/2]]) / (1-nu ** 2)
```